In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [5]:
nodeslist = pd.read_json('nodeslist.json')
nodesDF = nodeslist[~nodeslist['hasChild']]
nodesDF = nodesDF[nodesDF['value']!='']
nodesDF.shape

(1299, 3)

In [6]:
nodesDF.head()

,hasChild,label,value
3,False,Air Conditioners or Heat Pumps,HouseholdItems/Appliances/AirConditionersHeatP...
4,False,Blenders,HouseholdItems/Appliances/Blenders
5,False,Food Processors,HouseholdItems/Appliances/FoodProcessors
6,False,Mixers,HouseholdItems/Appliances/Mixers
7,False,Other Large Appliances,HouseholdItems/Appliances/OtherLargeAppliances


In [7]:
home = 'https://www.seattle.gov/util/MyServices/WhereDoesItGo/'
my_resp = lambda x: requests.get(home+x+'/index.htm', timeout=5)
my_content = lambda x: BeautifulSoup(x.content, "html.parser")
my_h3 = lambda x: x.find_all('h3')[1:]
my_labels = lambda x: [s['class'][0] for s in x]

In [8]:
labels = []
for i in range(nodesDF.shape[0]):
    _resp = my_resp(nodesDF.iloc[i]['value'])
    _cont = my_content(_resp)
    _h3 = my_h3(_cont)
    try:
        _lab = my_labels(_h3)
        labels.append(_lab)
    except:
        labels.append([''])

In [9]:
compost = np.array([True if 'food' in s else False for s in labels])
garbage = np.array([True if 'garbage' in s else False for s in labels])
recycle = np.array([True if 'recycle' in s else False for s in labels])
binside = np.array([True if 'binSide' in s else False for s in labels])
donate  = np.array([True if 'donate' in s else False for s in labels])
transfer = np.array([True if 'transfer' in s else False for s in labels])

In [11]:
print(sum(compost)/len(compost))
print(sum(garbage)/len(compost))
print(sum(recycle)/len(compost))
print(sum(binside)/len(compost))
print(sum(donate)/len(compost))
print(sum(transfer)/len(compost))

0.1170130869899923
0.4973056197074673
0.27251732101616627
0.049268668206312545
0.11855273287143957
0.147036181678214


In [52]:
compost2 = compost.copy()

for i in range(len(compost2)):
    _w = nodesDF['label'].iloc[i]
    _w = _w.lower()
    if (('plastic' in _w)|('glass' in _w)|('pot' in _w)
       |('jug' in _w)|('cup' in _w)|('bowl' in _w)
       |('spoon' in _w)|('fork' in _w)|('knife' in _w)
       |('spork' in _w)):
        compost2[i] = False

In [55]:
print(sum(compost2)/len(compost))

0.08929946112394149


In [56]:
nodesDF['compost'] = compost2

In [15]:
nodesDF = nodesDF.drop(columns=['hasChild','value'])

In [43]:
nodesDF.head()

,label,compost
3,Air Conditioners or Heat Pumps,False
4,Blenders,False
5,Food Processors,False
6,Mixers,False
7,Other Large Appliances,False


In [57]:
nodesDF.to_json('compost_or_not.json',orient='records')